In [ ]:
%pip install tensorflow==2.12.0 python_dotenv wandb tensorflow-addons==0.22.0 # tensorboard_plugin_profile==2.8.0

In [3]:
from google.colab import drive

drive.mount("/content/drive")

from google.colab import auth

auth.authenticate_user()

[Errno 2] No such file or directory: '/content/drive/MyDrive/Mushroom-Classifier'
/home/broug/Desktop/Mushroom-Classifier/training


In [2]:
%cd /content/drive/MyDrive/Mushroom-Classifier

%load_ext autoreload
%autoreload 2

In [1]:
import math, re, os, pickle
from datetime import datetime
import tensorflow as tf
from tensorflow.keras import layers, Model, Sequential
import tensorflow_addons as tfa
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split
from src.models.swintransformer import SwinTransformer
import wandb
from wandb.keras import WandbCallback, WandbMetricsLogger, WandbEvalCallback
from train_config import CFG

print(f"Tensorflow version {tf.__version__}")
AUTO = tf.data.experimental.AUTOTUNE

CFG = CFG()

2023-11-12 13:09:51.653571: I tensorflow/compiler/xla/stream_executor/tpu/tpu_initializer_helper.cc:242] Libtpu path is: libtpu.so
D1112 13:09:51.762660973   83890 config.cc:175]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D1112 13:09:51.762681117   83890 config.cc:175]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D1112 13:09:51.762685646   83890 config.cc:175]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D1112 13:09:51.762689146   83890 config.cc:175]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D1112 13:09:51.762693623   83890 config.cc:175]                        gRPC EXPERIMENT unconstrained_max_quota_buffer_size OFF (default:OFF)
D1112 13:09:51.762697499   83890 config.cc:175]                        gRPC EXPERIMENT event_engine_client                 OFF (default:OFF)
D1112 13:09:51.76270166

Tensorflow version 2.14.0


In [2]:
np.random.seed(CFG.SEED)
tf.random.set_seed(CFG.SEED)
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
os.environ["PYTHONHASHSEED"] = str(CFG.SEED)

In [ ]:
# Detect hardware
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
except ValueError:  # If TPU not found
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()
REPLICAS = strategy.num_replicas_in_sync
print("Number of accelerators: ", strategy.num_replicas_in_sync)

## Visualization Utils


In [6]:
def batch_to_numpy_images_and_labels(data):
    images, labels = data
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()
    if numpy_labels.dtype == object: # binary string in this case, these are image ID strings
        numpy_labels = [None for _ in enumerate(numpy_images)]
    # If no labels, only image IDs, return None for labels (this is the case for test data)
    return numpy_images, numpy_labels

def title_from_label_and_target(label, correct_label):
    if correct_label is None:
        return class_dict[label], True
    correct = (label == correct_label)
    return "{} [{}{}{}]".format(class_dict[label], 'OK' if correct else 'NO', u"\u2192" if not correct else '',
                                class_dict[correct_label] if not correct else ''), correct

def display_one_flower(image, title, subplot, red=False, titlesize=16):
    image = (image - image.min()) / (
        image.max() - image.min()
    )  # convert to [0, 1] for avoiding matplotlib warning
    plt.subplot(*subplot)
    plt.axis('off')
    plt.imshow(image)
    if len(title) > 0:
        plt.title(title, fontsize=int(titlesize) if not red else int(titlesize/1.2), color='red' if red else 'black', fontdict={'verticalalignment':'center'}, pad=int(titlesize/1.5))
    return (subplot[0], subplot[1], subplot[2]+1)

def display_batch_of_images(databatch, predictions=None):
    """This will work with:
    (images), (images, predictions), ((images, labels)), ((images, labels), predictions)
    """
    # data
    images, labels = batch_to_numpy_images_and_labels(databatch)
    if labels is None:
        labels = [None for _ in enumerate(images)]

    # auto-squaring: this will drop data that does not fit into square or square-ish rectangle
    rows = int(math.sqrt(len(images)))
    cols = len(images)//rows

    # size and spacing
    FIGSIZE = 13.0
    SPACING = 0.1
    subplot=(rows,cols,1)
    if rows < cols:
        plt.figure(figsize=(FIGSIZE,FIGSIZE/cols*rows))
    else:
        plt.figure(figsize=(FIGSIZE/rows*cols,FIGSIZE))

    # display
    for i, (image, label) in enumerate(zip(images[:rows*cols], labels[:rows*cols])):
        title = '' if label is None else class_dict[label]
        correct = True
        if predictions is not None:
            title, correct = title_from_label_and_target(predictions[i], label)
        dynamic_titlesize = FIGSIZE*SPACING/max(rows,cols)*40+3 # magic formula tested to work from 1x1 to 10x10 images
        subplot = display_one_flower(image, title, subplot, not correct, titlesize=dynamic_titlesize)

    #layout
    plt.tight_layout()
    if label is None and predictions is None:
        plt.subplots_adjust(wspace=0, hspace=0)
    else:
        plt.subplots_adjust(wspace=SPACING, hspace=SPACING)
    plt.show()

def display_confusion_matrix(cmat, score, precision, recall):
    plt.figure(figsize=(15,15))
    ax = plt.gca()
    ax.matshow(cmat, cmap='Reds')
    ax.set_xticks(range(len(class_dict)))
    ax.set_xticklabels(class_dict, fontdict={'fontsize': 7})
    plt.setp(ax.get_xticklabels(), rotation=45, ha="left", rotation_mode="anchor")
    ax.set_yticks(range(len(class_dict)))
    ax.set_yticklabels(class_dict, fontdict={'fontsize': 7})
    plt.setp(ax.get_yticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    titlestring = ""
    if score is not None:
        titlestring += 'f1 = {:.3f} '.format(score)
    if precision is not None:
        titlestring += '\nprecision = {:.3f} '.format(precision)
    if recall is not None:
        titlestring += '\nrecall = {:.3f} '.format(recall)
    if titlestring != "":
        ax.text(101, 1, titlestring, fontdict={'fontsize': 18, 'horizontalalignment':'right', 'verticalalignment':'top', 'color':'#804040'})
    plt.show()

def display_training_curves(training, validation, title, subplot):
    if subplot%10==1: # set up the subplots on the first call
        plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title(f'model {title}')
    ax.set_ylabel(title)
    #ax.set_ylim(0.28,1.05)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])

In [7]:
def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1))
         for filename in filenames]
    return np.sum(n)

def decode_image(image_data, smallest_side, CFG):
    image = tf.image.decode_jpeg(image_data, channels=3)  # image format uint8 [0,255]
    image = tf.cast(image, tf.float32)
    image = tf.image.resize_with_crop_or_pad(image, smallest_side, smallest_side)
    image = tf.image.resize(image, size=CFG.RAW_SIZE, method="lanczos5")
    image = tf.image.random_crop(image, size=[*CFG.CROP_SIZE, 3])  #, method="lanczos5"
    return image

def read_labeled_tfrecord(example, CFG):
    feature_description = {
        "image/encoded": tf.io.FixedLenFeature([], tf.string),
        "image/id": tf.io.FixedLenFeature([], tf.string),
        "image/meta/width": tf.io.FixedLenFeature([], tf.int64),
        "image/meta/height": tf.io.FixedLenFeature([], tf.int64),
        "image/class/label": tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, feature_description)

    width = tf.cast(example['image/meta/width'], tf.int32)
    height = tf.cast(example['image/meta/height'], tf.int32)
    smallest_side = tf.minimum(width, height)

    image = decode_image(example["image/encoded"], smallest_side, CFG)
    label = tf.cast(example["image/class/label"], tf.int32)

    return image, label
    # id = tf.cast(example['image/id'], tf.string)
    # return image, label, id

def load_dataset(filenames, CFG):
  # read from TFRecords. For optimal performance, read from multiple
  # TFRecord files at once and set the option experimental_deterministic = False
  # to allow order-altering optimizations.
  option_no_order = tf.data.Options()
  option_no_order.experimental_deterministic = False

  dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
  dataset = dataset.with_options(option_no_order)

  dataset = dataset.map(lambda x: read_labeled_tfrecord(x, CFG), num_parallel_calls=AUTO)
  return dataset

def basic_augment(image, prob_hflip=0.5, prob_vflip=0.5, prob_color_jitter=0.5, prob_rotate=0.5):
    """Apply various augmentations to an image.

    Args:
        image: The input image.
        prob_hflip: Probability of applying horizontal flip.
        prob_vflip:
        prob_color_jitter: Probability of applying color jitter.
        prob_rotate: Probability of applying rotation.

    Returns:
        Augmented image.
    """
    # Horizontal Flip
    if tf.random.uniform([]) < prob_hflip:
        image = tf.image.random_flip_left_right(image)

    if tf.random.uniform([]) < prob_vflip:
        image = tf.image.random_flip_up_down(image)

    # Color Jitter
    if tf.random.uniform([]) < prob_color_jitter:
        image = tf.image.random_brightness(image, max_delta=0.4)
        image = tf.image.random_contrast(image, lower=0.7, upper=1.3)
        image = tf.image.random_saturation(image, lower=0.7, upper=1.3)
        image = tf.image.random_hue(image, max_delta=0.3)

    # Rotation
    if tf.random.uniform([]) < prob_rotate:
        angles = tf.random.uniform([], -18, 18)
        image = tfa.image.rotate(image, angles, fill_mode='reflect')
    return image

def random_masking(image, randmask_prob=0.5):
    if tf.random.uniform([]) < randmask_prob:
        original_shape = tf.shape(image)

        count = tf.random.uniform([], 1, 11)

        erase_size = tf.random.uniform([], 0.2, 0.5)
        erase_size = tf.math.divide(erase_size, tf.math.sqrt(count))
        erase_value = tf.random.uniform([], 0., 255.)

        mask1 = int(erase_size * float(original_shape[0])) - (int(erase_size * float(original_shape[0])) % 2)
        mask2 = int(erase_size * float(original_shape[1])) - (int(erase_size * float(original_shape[1])) % 2)
        for k in tf.range(count):
            image = tfa.image.random_cutout(
                tf.expand_dims(image, axis=0),
                mask_size=(mask1, mask2),
                constant_values=erase_value
            )
            image = tf.squeeze(image, 0)
    return image

def augment(images, labels):
    images = tf.map_fn(basic_augment, images)
    images = tf.map_fn(random_masking, images)
    return images, labels

def get_batched_dataset(filenames, CFG, train=False):
    dataset = load_dataset(filenames, CFG)
    # dataset = dataset.cache() # This dataset fits in RAM
    dataset = dataset.repeat()
    dataset = dataset.batch(CFG.BATCH_SIZE, drop_remainder=True)
    if train:
        dataset = dataset.map(augment, num_parallel_calls=AUTO)
        # dataset = dataset.shuffle(BATCH_SIZE * 10)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

In [8]:
CFG.BATCH_SIZE = CFG.BASE_BATCH_SIZE * REPLICAS

GCS_PATH_SELECT = {
    192: f"gs://{CFG.GCS_REPO}/tfrecords-jpeg-192x192",
    224: f"gs://{CFG.GCS_REPO}/tfrecords-jpeg-224x224",
    331: f"gs://{CFG.GCS_REPO}/tfrecords-jpeg-331x331",
    512: f"gs://{CFG.GCS_REPO}/tfrecords-jpeg-512x512",
    None: f"gs://{CFG.GCS_REPO}/tfrecords-jpeg-raw",
}
GCS_PATH = GCS_PATH_SELECT[None]

filenames = tf.io.gfile.glob(f"{GCS_PATH}/train*.tfrec")
filenames, test_filenames = train_test_split(filenames, test_size=1, shuffle=True)
training_filenames, validation_filenames = train_test_split(
    filenames, test_size=0.15, shuffle=True
)

num_train = count_data_items(training_filenames)
num_val = count_data_items(validation_filenames)
num_test = count_data_items(test_filenames)

validation_steps = num_val / CFG.BATCH_SIZE // REPLICAS
steps_per_epoch = num_train / CFG.BATCH_SIZE // REPLICAS
TOTAL_STEPS = int(steps_per_epoch * (CFG.EPOCHS - 1))

class_dict = pickle.load(open("src/class_dict_NEW.pkl", "rb"))

In [ ]:
if CFG.DEBUG:
    print("Training data shapes:")
    for (image, meta), label in get_batched_dataset(
        training_filenames, CFG, train=True
    ).take(3):
        print(image.numpy().shape, label.numpy().shape)
    print("Training data label examples:", label.numpy())
    print("Validation data shapes:")
    for (image, meta), label in get_batched_dataset(validation_filenames, CFG).take(3):
        print(image.numpy().shape, label.numpy().shape)
    print("Validation data label examples:", label.numpy())

In [42]:
if CFG.DEBUG:
    # Peek at training data
    training_dataset = get_batched_dataset(training_filenames, CFG, train=True)
    training_dataset = training_dataset.unbatch().batch(20)
    train_batch = iter(training_dataset)

In [ ]:
if CFG.DEBUG:
    # run this cell again for next set of images
    display_batch_of_images(next(train_batch))

In [23]:
with strategy.scope():
    img_adjust_layer = layers.Lambda(lambda data: tf.keras.applications.imagenet_utils.preprocess_input(tf.cast(data, tf.float32), mode="torch"), input_shape=[*CFG.CROP_SIZE, 3])
    pretrained_model = SwinTransformer(CFG.MODEL, num_classes=len(class_dict), include_top=False, pretrained=True, use_tpu=True)

    model = Sequential([
        img_adjust_layer,
        pretrained_model,
    ])

    # Assuming input_image is the input layer for model
    input_image = layers.Input(shape=model.input_shape[1:])
    model_output = model(input_image)
    output = layers.Dense(len(class_dict), activation='softmax')(model_output)

    # Create the final model
    final_model = Model(inputs=input_image, outputs=output)

    top3 = tf.keras.metrics.SparseTopKCategoricalAccuracy(3, name='top-3-accuracy')

final_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5, epsilon=1e-8),
    loss = 'sparse_categorical_crossentropy',
    metrics=['accuracy', top3]
)
final_model.summary()

Training data shapes:


(256, 224, 224, 3) (256,)
(256, 224, 224, 3) (256,)
(256, 224, 224, 3) (256,)


2023-11-12 13:39:16.484093: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


Training data label examples: [100 215  58 ... 352 187  18]
Validation data shapes:
(256, 224, 224, 3) (256,)
(256, 224, 224, 3) (256,)
(256, 224, 224, 3) (256,)
Validation data label examples: [  8 233 457 ... 208 206 197]


2023-11-12 13:39:17.991477: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [11]:
def get_lr_callback(batch_size=8, plot=False):
    lr_start   = 0.000001
    lr_max     = 0.00000030 * batch_size
    lr_min     = 0.0000001
    lr_ramp_ep = 5
    lr_sus_ep  = 0
    lr_decay   = 0.9
    CFG.LR_START, CFG.LR_MAX, CFG.LR_MIN, CFG.RAMPEP, CFG.DECAY = lr_start, lr_max, lr_min, lr_ramp_ep, lr_decay
    
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start

        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max

        elif CFG.SCHEDULER=='exp':
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min

        elif CFG.SCHEDULER=='cosine':
            decay_total_epochs = CFG.EPOCHS - lr_ramp_ep - lr_sus_ep + 3
            decay_epoch_index = epoch - lr_ramp_ep - lr_sus_ep
            phase = math.pi * decay_epoch_index / decay_total_epochs
            cosine_decay = 0.4 * (1 + math.cos(phase))
            lr = (lr_max - lr_min) * cosine_decay + lr_min
        
        gc.collect()
        return lr
    if plot:
        plt.figure(figsize=(10,5))
        plt.plot(np.arange(CFG.EPOCHS), [lrfn(epoch) for epoch in np.arange(CFG.EPOCHS)], marker='o')
        plt.xlabel('epoch'); plt.ylabel('learning rate')
        plt.title('Learning Rate Scheduler')
        plt.show()

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=False)
    return lr_callback

_=get_lr_callback(CFG.BATCH_SIZE, plot=True )

In [37]:
config = wandb.helper.parse_config(
    CFG, include=('AUGMENT', 'BATCH_SIZE', 'EPOCHS', 'IMAGE_SIZE', 'LR_START', 'LR_MAX', 'LR_MIN', 'LR_RAMPEP', 'LR_DECAY', 'MODEL_SIZE')
)
wandb.init(
    project="Mushroom-Classifier",
    tags=[CFG.MODEL, CFG.OPT, 'ExponentialWarmup', str(CFG.CROP_SIZE[0])],
    config=config,
    dir="../",
)

In [12]:
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
options = tf.saved_model.SaveOptions(
        experimental_io_device="/job:localhost"
    )
CFG.CKPT_DIR.mkdir(parents=True, exist_ok=True)
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=CFG.ES_PATIENCE,
        verbose=1,
        mode="max",
        restore_best_weights=True,
    ),
    wandb.keras.WandbMetricsLogger(log_freq="batch"),
    wandb.keras.WandbModelCheckpoint(
        str(CFG.CKPT_DIR),  # .h5 for weights, dir for whole model
        monitor="val_accuracy",
        mode="max",
        verbose=1,
        save_best_only=True,
        save_weights_only=False,
        options=options,
        initial_value_threshold=0.75,
    ),
    get_lr_callback(CFG.BATCH_SIZE),

    # tf.keras.callbacks.TensorBoard(
    #     log_dir = logs,
    #     histogram_freq = 1,
    #     # profile_batch = '500,520'
    # )
]

2023-11-12 13:10:34.247467: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-12 13:10:34.296752: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2023-11-12 13:10:35.799623: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-11-12 13:10:35.799686: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.


In [21]:
history = final_model.fit(
    get_batched_dataset(training_filenames, CFG),
    steps_per_epoch=steps_per_epoch,
    epochs=CFG.EPOCHS,
    validation_data=get_batched_dataset(validation_filenames, CFG),
    validation_steps=validation_steps,
    callbacks=callbacks,
)

Epoch 1/30
47/47 [==============================] - 44s 935ms/step - loss: 5.9447 - sparse_categorical_accuracy: 0.0048 - sparse_top_3_categorical_accuracy: 0.0125 - val_loss: 6.1053 - val_sparse_categorical_accuracy: 0.0000e+00 - val_sparse_top_3_categorical_accuracy: 0.0098
Epoch 2/30
47/47 [==============================] - 42s 900ms/step - loss: 5.9466 - sparse_categorical_accuracy: 0.0049 - sparse_top_3_categorical_accuracy: 0.0137 - val_loss: 6.0991 - val_sparse_categorical_accuracy: 0.0039 - val_sparse_top_3_categorical_accuracy: 0.0098
Epoch 3/30
47/47 [==============================] - 42s 893ms/step - loss: 5.9388 - sparse_categorical_accuracy: 0.0052 - sparse_top_3_categorical_accuracy: 0.0125 - val_loss: 6.1144 - val_sparse_categorical_accuracy: 0.0000e+00 - val_sparse_top_3_categorical_accuracy: 0.0020
Epoch 4/30
47/47 [==============================] - 41s 882ms/step - loss: 5.9388 - sparse_categorical_accuracy: 0.0050 - sparse_top_3_categorical_accuracy: 0.0152 - val_los

In [ ]:
wandb.finish()

In [ ]:
import pandas as pd


def process_and_save_predictions(model, dataset):
    """Process a dataset and save the predictions to a CSV file.
    Args:
    - model: The trained model used for predictions.
    - preprocessing_function: Function that preprocesses and returns a dataset from TFRecord files.
    - tfrecord_filenames: List of TFRecord filenames to process.

    Returns:
    - A Pandas DataFrame with columns for Image ID, Prediction, and Actual Label.
    """
    predictions = []
    actual_labels = []
    image_ids = []

    for (image_data, metadata), label, id in dataset:
        # Predict
        pred = model.predict((image_data, metadata))
        pred_label = pred.argmax(axis=1)  # Get the class with the highest probability

        # Extract image ID - adjust this based on your metadata structure
        image_id = (
            id  # This line needs to be adjusted to your specific metadata structure
        )

        # Append to lists
        predictions.extend(pred_label)
        actual_labels.extend(
            label.numpy()
        )  # Label is already sparse, no need for argmax
        image_ids.extend(
            image_id.numpy()
        )  # Adjust if image_id extraction method is different

    # Create DataFrame
    results_df = pd.DataFrame(
        {
            "Image ID": image_ids,
            "Prediction": predictions,
            "Actual Label": actual_labels,
        }
    )
    results_df.to_csv("data/predictions.csv", index=False)

    return results_df

In [16]:
run = wandb.init()
artifact = run.use_artifact(
    "g-broughton/Mushroom-Classifier/run_fjjcu05u_model:v11", type="model"
)
artifact_dir = artifact.download()

In [ ]:
ds = load_dataset(validation_filenames, CFG)
ds = (
    ds.map(
        lambda x, y, z: ((x, process_meta(y)), z, y["id"]),
        num_parallel_calls=tf.data.AUTOTUNE,
    )
    .batch(512)
    .prefetch(tf.data.AUTOTUNE)
)
df = process_and_save_predictions(final_model, ds)

In [ ]:
wrong = df[df["Prediction"] != df["Actual Label"]]
df["class"] = df.apply(lambda x: class_dict[x["Actual Label"]], axis=1)
wrong["class"].value_counts()

In [ ]:
df = df.iloc[~wrong.index]
df["class"].value_counts()